# 贪婪和恐慌指数

什么是恐慌和贪婪指数？

恐慌和贪婪指数(Fear and Greed Index)是衡量比特币市场情绪的常用指标。该指数综合了以下维度：波动性(25%权重)，市场动能/成交量(25%)，社交媒体情绪(15%)，投资者调研(15%)，BTC市值占比(10%)，比特币谷歌搜索趋势(10%)。指数的取值范围是0-100，解读如下：

- 0-25: 极度恐慌
- 25-45: 恐慌
- 45-55: 中性
- 55-75: 贪婪 
- 75-100: 极度贪婪

恐慌和贪婪指数有哪些优缺点？

优点：
- 取值范围是标准化区间，容易解读
- 能够相对有效的衡量市场情绪

缺点：
- 无法用于市场择时（market timing）
- 牛市时期可能长期维持极度贪婪,但价格仍持续上涨
- 熊市时期可能持续处于恐慌区间,但价格仍继续下跌
- 指数极值并不必然意味着价格拐点

研究目标：探索恐慌和贪婪指数在市场择时方面的实用价值。

## 数据

- 从官方获取恐慌和贪婪指数
- 从雅虎财经获取比特币历史价格
- 读取数据，清洗和合并数据

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from indicators import lowpass_filter, peak, valley

In [2]:
# fear and greed index
fgi = pd.read_csv("../data/fear_greed_index.csv", index_col=0, parse_dates=True)

# btcusd
btc = pd.read_csv("../data/yahoo/Bitcoin.csv", index_col=0, parse_dates=True)

# merge data
df = (
    fgi.join(btc["Close"], how="left")
    .rename(columns={"Close": "btcusd", "value": "fgi"})
    .dropna()
)

df

,fgi,classification,btcusd
date,,,
2018-02-01,30.0,Fear,9170.540039
2018-02-02,15.0,Extreme Fear,8830.750000
2018-02-03,40.0,Fear,9174.910156
2018-02-04,24.0,Extreme Fear,8277.009766
2018-02-05,11.0,Extreme Fear,6955.270020
...,...,...,...
2025-02-03,44.0,Fear,101405.421875
2025-02-04,72.0,Greed,97871.820312
2025-02-05,54.0,Neutral,96615.445312


In [3]:
data = df.copy()
data["smooth_fgi"] = lowpass_filter(data["fgi"], 10)

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["fgi"], name="Fear greed index"))
fig.add_trace(
    go.Scatter(x=data.index, y=data["smooth_fgi"], name="Smoothed index(10-day)")
)
fig.update_layout(
    title="Fear and greed index",
    width=1000,
    height=600,
    legend=dict(
        orientation="h", xanchor="center", yanchor="bottom", x=0.5, y=1.02, title=""
    ),
    xaxis_title="Date",
    yaxis_title="Index(0-100)",
)
fig.show()

## 实验1

恐慌和贪婪指数是否能识别比特币的长期顶部和底部？

我们提出以下方法：
- 统计市场处于恐慌或极度恐慌的连续天数
- 统计市场处于贪婪或极度贪婪的连续天数
- 分析这两个衍生指标与比特币长期顶部和底部的关系

In [4]:
data = df.copy()

fear_days = np.zeros(len(df))
greed_days = np.zeros(len(df))

for i in range(1, len(fear_days)):
    if data["classification"].iloc[i] in ["Fear", "Extreme Fear"]:
        fear_days[i] = fear_days[i - 1] + 1
    if data["classification"].iloc[i] in ["Greed", "Extreme Greed"]:
        greed_days[i] = greed_days[i - 1] + 1

data["fear_days"] = fear_days
data["greed_days"] = greed_days

data["fear_days_peak"] = peak(data["fear_days"])
data["greed_days_peak"] = peak(data["greed_days"])

data

,fgi,classification,btcusd,fear_days,greed_days,fear_days_peak,greed_days_peak
date,,,,,,,
2018-02-01,30.0,Fear,9170.540039,0.0,0.0,0,0
2018-02-02,15.0,Extreme Fear,8830.750000,1.0,0.0,0,0
2018-02-03,40.0,Fear,9174.910156,2.0,0.0,0,0
2018-02-04,24.0,Extreme Fear,8277.009766,3.0,0.0,0,0
2018-02-05,11.0,Extreme Fear,6955.270020,4.0,0.0,0,0
...,...,...,...,...,...,...,...
2025-02-03,44.0,Fear,101405.421875,1.0,0.0,1,0
2025-02-04,72.0,Greed,97871.820312,0.0,1.0,0,1
2025-02-05,54.0,Neutral,96615.445312,0.0,0.0,0,0


连续恐慌天数和比特币价格。

In [5]:
# 定义配色方案
COLORS = {
    "bitcoin": "#F7931A",  # 比特币标准色
    "fear": "#FF6B6B",  # 恐慌指数颜色
    "grid": "#E5E5E5",  # 网格线颜色
    "reference": "#34495e",  # 参考线颜色
}

fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=(
        "<b>Bitcoin Price (USD)</b>",
        "<b>Cumulative Days in Fear</b>",
    ),
    shared_xaxes=True,
    vertical_spacing=0.12,
)

# Bitcoin price chart
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["btcusd"],
        name="Bitcoin Price",
        line=dict(color=COLORS["bitcoin"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Price</b>: $%{y:,.0f}<br><extra></extra>",
    ),
    row=1,
    col=1,
)

# Cumulative fear days chart
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["fear_days"],
        name="Fear Days",
        line=dict(color=COLORS["fear"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Days in Fear</b>: %{y:.0f}<br><extra></extra>",
    ),
    row=2,
    col=1,
)

# Add reference lines for fear levels
reference_levels = [50, 100, 150]
for level in reference_levels:
    fig.add_hline(
        y=level,
        row=2,
        col=1,
        line_dash="dash",
        line_color=COLORS["reference"],
        line_width=1,
        annotation_text=f"{level} days",
        annotation_position="right",
    )

# Highlight fear peaks
peak_dates = data.query("fear_days >= 50 & fear_days_peak == 1").index
for d in peak_dates:
    fig.add_vline(
        x=d, line_dash="dot", line_color=COLORS["reference"], line_width=1, opacity=0.7
    )

# Update layout
fig.update_layout(
    template="plotly_white",  # 使用白色主题
    width=1200,
    height=800,
    title=dict(
        text="<b>Bitcoin Price vs Cumulative Days in Fear</b>",
        x=0.5,
        y=0.95,
        font=dict(size=20),
    ),
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x unified",
)

# Update axes
fig.update_yaxes(
    type="log",
    row=1,
    col=1,
    title="Price (USD, log scale)",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

fig.update_yaxes(
    row=2,
    col=1,
    title="Cumulative Days",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

fig.update_xaxes(gridcolor=COLORS["grid"], showgrid=True)

fig.show()

连续贪婪天数和比特币价格

In [6]:
# 定义配色方案
COLORS = {
    "bitcoin": "#F7931A",  # 比特币标准色
    "fear": "#FF6B6B",  # 恐慌指数颜色
    "grid": "#E5E5E5",  # 网格线颜色
    "reference": "#34495e",  # 参考线颜色
}

fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=(
        "<b>Bitcoin Price (USD)</b>",
        "<b>Cumulative Days in Greed</b>",
    ),
    shared_xaxes=True,
    vertical_spacing=0.12,
)

# Bitcoin price chart
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["btcusd"],
        name="Bitcoin Price",
        line=dict(color=COLORS["bitcoin"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Price</b>: $%{y:,.0f}<br><extra></extra>",
    ),
    row=1,
    col=1,
)

# Cumulative greed days chart
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["greed_days"],
        name="Greed Days",
        line=dict(color=COLORS["fear"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Days in Greed</b>: %{y:.0f}<br><extra></extra>",
    ),
    row=2,
    col=1,
)

# Add reference lines for greed levels
reference_levels = [40, 80, 100]
for level in reference_levels:
    fig.add_hline(
        y=level,
        row=2,
        col=1,
        line_dash="dash",
        line_color=COLORS["reference"],
        line_width=1,
        annotation_text=f"{level} days",
        annotation_position="right",
    )

# Highlight fear peaks
peak_dates = data.query("greed_days >= 40 & greed_days_peak == 1").index
for d in peak_dates:
    fig.add_vline(
        x=d, line_dash="dot", line_color=COLORS["reference"], line_width=1, opacity=0.7
    )

# Update layout
fig.update_layout(
    template="plotly_white",  # 使用白色主题
    width=1200,
    height=800,
    title=dict(
        text="<b>Bitcoin Price vs Cumulative Days in Greed</b>",
        x=0.5,
        y=0.95,
        font=dict(size=20),
    ),
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x unified",
)

# Update axes
fig.update_yaxes(
    type="log",
    row=1,
    col=1,
    title="Price (USD, log scale)",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

fig.update_yaxes(
    row=2,
    col=1,
    title="Cumulative Days",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

fig.update_xaxes(gridcolor=COLORS["grid"], showgrid=True)

fig.show()

基于图表分析，我们可以得出以下结论：

- 累计恐慌天数的极值可能达到50天，100天或150天，但是无法识别市场底部
- 当累计贪婪天数超过80天时，市场通常会接近周期性顶部
- 虽然与绝对高点存在时间差，但这种预警信号的准确性较高
- 市场情绪如同钟摆，不会永远停留在极端状态
- 历史数据显示，极度乐观情绪难以持续超过80-100天

对投资策略的启示：

- 当累计贪婪天数达到80天以上时，长期投资者应考虑适度减仓
- 将这个指标与原始恐慌和贪婪指数结合使用，能够更好的控制风险

## 实验2

恐慌贪婪指数能否识别比特币的中期高点和低点？

我们将中期定义为1-3个月。

我们采用以下方法：
- 将菲舍尔转换应用于恐慌和贪婪指数，将其标准化
- 将“标准化”的指数跟价格进行比较

将输出序列理解为恐慌和贪婪程度的“相对变化”，当菲舍尔转换超过+2时，可以理解为市场情绪的改善过于“贪婪”，这可能与局部高点相对应，当菲舍尔转换超过-2时，可以理解为市场情绪的恶化过于“恐慌”，这可能与局部低点相对应。

什么是菲舍尔转换？

菲舍尔转换的核心思想：将任意概率分布转化为近似正态分布。菲舍尔转换在时间序列分析中的应用：

1. 将时间序列转换为近似服从正态分布
2. 识别极值（局部高点和低点）
3. 输出结果的取值范围通常是[-3, 3]，超过+/-2时通常表示极值，穿越零线可能暗示趋势转变
4. 主要功能是标准化数据并识别极值，而不是分解时间序列的周期成分

In [7]:
def find_trend_periods(series: pd.Series) -> list:
    periods = []
    start = None

    for i in range(len(series)):
        if series.iloc[i] == 1 and start is None:
            start = series.index[i]
        elif series.iloc[i] == 0 and start is not None:
            end = series.index[i - 1]
            periods.append((start, end))
            start = None

    if start is not None:
        end = series.index[-1]
        periods.append((start, end))

    return periods


def fisher_transform(series: pd.Series, period: int = 10) -> pd.Series:
    highest = series.rolling(period, min_periods=1).max()
    lowest = series.rolling(period, min_periods=1).min()
    values = np.zeros(len(series))
    fishers = np.zeros(len(series))

    for i in range(1, len(series)):
        values[i] = (
            0.66
            * (
                (series.iloc[i] - lowest.iloc[i]) / (highest.iloc[i] - lowest.iloc[i])
                - 0.5
            )
            + 0.67 * values[i - 1]
        )
        values[i] = max(min(values[i], 0.999), -0.999)
        fishers[i] = (
            0.5 * np.log((1 + values[i]) / (1 - values[i])) + 0.5 * fishers[i - 1]
        )

    return pd.Series(fishers, index=series.index)

In [8]:
data = df.copy()

# period of fisher transform
fisher_period = 10

# apply fisher transform to fear index
data["fisher"] = fisher_transform(data["fgi"], period=fisher_period)

# identify periods of fisher above/below extreme levels
peak_periods = find_trend_periods(data["fisher"] > 2)
valley_periods = find_trend_periods(data["fisher"] < -2)

# data

In [9]:
# 定义配色方案
COLORS = {
    "btc": "#F7931A",  # 比特币标准色
    "fisher": "#553C9A",  # 菲舍尔转换曲线颜色
    "peak": "#FF6B6B",  # 极值区域(红色)
    "valley": "#38A169",  # 极值区域(绿色)
    "grid": "#E5E5E5",  # 网格线颜色
    "reference": "#718096",  # 参考线颜色
}

# 创建子图
fig = make_subplots(
    rows=2,
    cols=1,
    vertical_spacing=0.12,
    shared_xaxes=True,
    subplot_titles=(
        "<b>Bitcoin Price (USD)</b>",
        "<b>Fisher Transform of Fear and Greed index</b>",
    ),
)

# 比特币价格图表
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["btcusd"],
        name="BTC/USD",
        line=dict(color=COLORS["btc"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Price</b>: $%{y:,.0f}<br><extra></extra>",
    ),
    row=1,
    col=1,
)

# 添加极值区域背景
for x0, x1 in peak_periods:
    fig.add_vrect(
        x0=x0,
        x1=x1,
        fillcolor=COLORS["peak"],
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )

for x0, x1 in valley_periods:
    fig.add_vrect(
        x0=x0,
        x1=x1,
        fillcolor=COLORS["valley"],
        opacity=0.2,
        line_width=0,
        row=1,
        col=1,
    )

# 菲舍尔转换图表
fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["fisher"],
        name="Fisher Transform",
        line=dict(color=COLORS["fisher"], width=2),
        hovertemplate="<b>Date</b>: %{x}<br>"
        + "<b>Fisher</b>: %{y:.2f}<br><extra></extra>",
    ),
    row=2,
    col=1,
)

# 添加参考线
reference_levels = [-3, -2, 2, 3]
for level in reference_levels:
    fig.add_hline(
        y=level,
        line_dash="dash",
        line_color=COLORS["reference"],
        line_width=1,
        row=2,
        col=1,
    )

# 更新布局
fig.update_layout(
    template="plotly_white",
    width=1200,
    height=800,
    title=dict(
        text="<b>Bitcoin Price and Market Sentiment Analysis</b>",
        x=0.5,
        y=0.95,
        font=dict(size=20),
    ),
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x unified",
)

# 更新坐标轴
fig.update_yaxes(
    row=1,
    col=1,
    title="Price (USD)",
    type="log",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

fig.update_yaxes(
    row=2,
    col=1,
    title="Fisher Transform Value",
    gridcolor=COLORS["grid"],
    title_font=dict(size=14),
)

# 添加时间范围选择器
fig.update_xaxes(
    rangeselector=dict(
        buttons=list(
            [
                dict(count=6, label="6M", step="month", stepmode="backward"),
                dict(count=1, label="1Y", step="year", stepmode="backward"),
                dict(count=2, label="2Y", step="year", stepmode="backward"),
                dict(count=5, label="5Y", step="year", stepmode="backward"),
                dict(step="all", label="All"),
            ]
        ),
        bgcolor="#E2E8F0",
        activecolor="#CBD5E0",
    ),
    gridcolor=COLORS["grid"],
    row=1,
    col=1,
)

fig.show()

如上图所示，菲舍尔转换在[-3,3]之间波动，呈现明显的均值回归特征，可以理解为市场情绪的“相对变化”。

基于图表分析，我们得到以下结论：

- 在横盘震荡时期，菲舍尔转换能够相对准确的识别中期高点和低点，典型案例：2024年5月-10月。
- 在趋势行情，指标的预测作用不那么有效，在长期熊市中，当菲舍尔转换高于+2，可以识别中期反弹高点，但是低于-2却无法识别中期底部，价格既可能反弹也可能继续下跌。长期牛市可以得到相似的结论。
- 简单来说，经过菲舍尔转换的指数能够识别中期高点和低点，但必须和其他市场结构指标结合使用。